In [1]:
import pandas as pd
import numpy as np
import altair as alt

# EDA

In [2]:
df = pd.read_csv('data/gradeguide_data.csv')
df.shape

(5948, 28)

In [3]:
df.head()

,Term,School Code,School,Department Name,Course Subject,Course Number,Course Title,Total Grades,GPA,GPA with 4.3 A+,...,C-,D+,D,D-,E,Desire to take,Understanding,Workload,Expectations,Increased Interest
0,FA 2015,LSA,"Literature, Science & the Arts",Political Science Department,POLSCI,498,Undergraduate Seminar in International Politics,40,3.632500,3.632500,...,1,0,0,0,0,78.0,90.0,38.0,61.0,84.0
1,FA 2017,MUS,"Music, Theatre & Dance","School of Music, Theatre and Dance",PIANO,111,Performance,51,3.662745,3.733333,...,0,0,0,0,0,60.0,100.0,20.0,96.0,82.0
2,FA 2014,LSA,"Literature, Science & the Arts",Economics Department,ECON,444,The European Economy,121,3.383471,3.395868,...,0,0,0,0,0,93.0,95.0,24.0,90.0,90.0
3,FA 2016,LSA,"Literature, Science & the Arts",Earth and Environmental Sciences,EARTH,222,Introductory Oceanography,108,3.508333,3.522222,...,0,0,0,0,1,50.0,88.0,8.0,79.0,61.0
4,WN 2017,BA,Ross School of Business,School of Business Administration,MKT,310,Fundamentals of Sales Management,118,3.540678,3.571186,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


Number of unique courses

In [4]:
df['Course'] = df['Course Subject'] + ' ' + df['Course Number'].astype(str)

In [5]:
df['Course'].nunique()

2141

In [6]:
df['Total Grades'].describe()

count    5948.000000
mean      111.740921
std       172.506769
min        17.000000
25%        39.000000
50%        58.000000
75%       104.000000
max      2152.000000
Name: Total Grades, dtype: float64

Number of unique courses by term

In [7]:
df[['Term','Course']].groupby('Term').nunique()

,Course
Term,
FA 2014,573
FA 2015,570
FA 2016,1095
FA 2017,1098
WN 2015,566
WN 2016,1005
WN 2017,1041


Data time span

In [8]:
df['Term'].value_counts()

FA 2017    1098
FA 2016    1095
WN 2017    1041
WN 2016    1005
FA 2014     573
FA 2015     570
WN 2015     566
Name: Term, dtype: int64

## 2 years of data: 2016 to 2018 (Winter 2016, Fall 2016, Winter 2017, Fall 2017)

In [9]:
df_16to18 = df[(df['Term'] != 'FA 2014') & (df['Term'] != 'FA 2015') & (df['Term'] != 'WN 2015')]

In [10]:
df_16to18['Term'].unique()

array(['FA 2017', 'FA 2016', 'WN 2017', 'WN 2016'], dtype=object)

## Calculate average GPA for each course in 4.0 scale

In [11]:
gpa_scale = pd.read_csv('data/umich_gpa_scale.csv')
letter_gpa_mapping = {k: v for (k, v) in zip(gpa_scale['Letter Grade'], gpa_scale['GPA'])}

In [12]:
df_16to18_courses = df_16to18.groupby('Course').sum().drop(columns = ['GPA', 'GPA with 4.3 A+', 'Desire to take', 'Understanding', 'Workload', 'Expectations', 'Increased Interest'])
df_16to18_courses.head()

,Course Number,Total Grades,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,E
Course,,,,,,,,,,,,,,,
AAS 103,103,37,3,12,4,3,6,4,0,4,0,0,1,0,0
AAS 104,312,118,2,26,39,22,11,11,3,2,1,1,0,0,0
AAS 111,444,337,15,93,80,61,38,24,7,12,5,1,0,0,1
AAS 208,208,32,1,4,7,4,4,7,0,1,3,1,0,0,0
AAS 248,248,34,0,14,12,4,3,1,0,0,0,0,0,0,0


In [13]:
def cal_gpa(row):
    student_count = row.to_dict()['Total Grades']
    total_gpa = sum([ row.to_dict()[l]*letter_gpa_mapping[l] for l in letter_gpa_mapping.keys()] )
    return total_gpa/student_count

In [14]:
df_16to18_courses['Average GPA'] = df_16to18_courses.apply(lambda x: cal_gpa(x), axis = 1)

Add back textual informations

In [15]:
df_16to18_courses.reset_index(inplace = True)
df_16to18_courses.drop(columns = ['Course Number'], inplace = True)

In [16]:
df_16to18_courses = df_16to18_courses.merge(
    df[['School Code', 'School', 'Department Name', 'Course Subject', 'Course Title', 'Course', 'Course Number']].drop_duplicates(subset=['Course']), on ='Course', how = 'inner', validate = 'one_to_many')

In [17]:
df_16to18_courses.head()

,Course,Total Grades,A+,A,A-,B+,B,B-,C+,C,...,D,D-,E,Average GPA,School Code,School,Department Name,Course Subject,Course Title,Course Number
0,AAS 103,37,3,12,4,3,6,4,0,4,...,1,0,0,3.310811,LSA,"Literature, Science & the Arts",Department of Afro-American and African Studies,AAS,First Year Social Science Seminar,103
1,AAS 104,118,2,26,39,22,11,11,3,2,...,0,0,0,3.436441,LSA,"Literature, Science & the Arts",Department of Afro-American and African Studies,AAS,First Year Humanities Seminar,104
2,AAS 111,337,15,93,80,61,38,24,7,12,...,0,0,1,3.436202,LSA,"Literature, Science & the Arts",Department of Afro-American and African Studies,AAS,Introduction to Africa and Its Diaspora,111
3,AAS 208,32,1,4,7,4,4,7,0,1,...,0,0,0,3.075000,LSA,"Literature, Science & the Arts",Department of Afro-American and African Studies,AAS,Introduction to African Art,208
4,AAS 248,34,0,14,12,4,3,1,0,0,...,0,0,0,3.685294,LSA,"Literature, Science & the Arts",Department of Afro-American and African Studies,AAS,"Crime, Race, and the Law",248


## Get course level

In [18]:
df_16to18_courses['Level'] = df_16to18_courses['Course Number'].apply(lambda x: 
    100 if x//100 == 1 else 
    200 if x//100 == 2 else 
    300 if x//100 == 3 else 
    400 if x//100 == 4 else 
    500 if x//100 == 5 else 
    600 if x//100 == 6 else 
    np.NaN
    )

In [19]:
df_16to18_courses['Level'].value_counts()

300.0    507
400.0    406
200.0    384
500.0    255
100.0    244
600.0    114
Name: Level, dtype: int64

## Weeder classes: lowerlevel courses with a high percentage of grades lower than C

Percentage of students who gets lower than C

In [20]:
def cal_weeded(row):
    student_count = row.to_dict()['Total Grades']
    total_A = sum([ row.to_dict()[l] for l in ['C-','D+','D','D-','E']] )
    return total_A/student_count*100

In [21]:
df_16to18_courses['weeded%'] = df_16to18_courses.apply(lambda x: cal_weeded(x), axis = 1)

In [22]:
df_16to18_courses['weeded%'].describe()

count    1926.000000
mean        1.711453
std         2.895608
min         0.000000
25%         0.000000
50%         0.212208
75%         2.377749
max        28.272251
Name: weeded%, dtype: float64

In [23]:
WEED_THRESHOLD = 5

In [24]:
df_16to18_courses[(df_16to18_courses['weeded%'] >= WEED_THRESHOLD)&
    (df_16to18_courses['Level'] <= 200)].shape[0] / df_16to18_courses[(df_16to18_courses['Level'] <= 200)].shape[0] 

0.1592356687898089

In [25]:
df_16to18_courses['Weeder'] = (df_16to18_courses['weeded%'] >= WEED_THRESHOLD)&(df_16to18_courses['Level'] <= 200)

### Easy As: classes with lots of As, and no lower than C grades

Percentage of students who gets an A+, A or A-

In [26]:
def cal_A(row):
    student_count = row.to_dict()['Total Grades']
    total_A = sum([ row.to_dict()[l] for l in ['A+','A','A-']] )
    return total_A/student_count*100

In [27]:
df_16to18_courses['A%'] = df_16to18_courses.apply(lambda x: cal_A(x), axis = 1)

In [28]:
EASY_THRESHOLD = 90

In [32]:
df_16to18_courses[(df_16to18_courses['A%'] >= EASY_THRESHOLD) & (df_16to18_courses['weeded%'] == 0)].shape[0] / df_16to18_courses.shape[0]

0.1526479750778816

In [30]:
df_16to18_courses['Easy A'] = (df_16to18_courses['A%'] >= EASY_THRESHOLD) & (df_16to18_courses['weeded%'] == 0)

In [37]:
final_df = df_16to18_courses[['Course', 'Course Title', 'School', 'School Code', 'Department Name', 'Course Subject', 'Course Number', 'Average GPA', 'Total Grades', 'Level', 'Weeder', 'weeded%', 'Easy A', 'A%']]

In [38]:
final_df.to_csv('data/course_gpa_16_to_18_cleaned.csv')